In [12]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
from google.colab import files

urls = [
    "https://csie.asia.edu.tw/zh_tw/TeacherIntroduction/Full_time_faculty",
    "https://csie.asia.edu.tw/zh_tw/TeacherIntroduction/Full_time_professor",
    "https://csie.asia.edu.tw/zh_tw/TeacherIntroduction/Full_time_Associate_Professor",
    "https://csie.asia.edu.tw/zh_tw/TeacherIntroduction/Full_time_Assistant_Professor",
    "https://csie.asia.edu.tw/zh_tw/TeacherIntroduction/Full_time_Lecturer",
    "https://csie.asia.edu.tw/zh_tw/TeacherIntroduction/Part_time_faculty"
]

all_teachers = []

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
    'Accept-Language': 'zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7',
}

for url in urls:
    if "Part_time" in url:
        teacher_type = "兼任教師"
    else:
        if "professor" in url.lower() and "associate" not in url.lower() and "assistant" not in url.lower():
            teacher_type = "專任教授"
        elif "associate" in url.lower():
            teacher_type = "專任副教授"
        elif "assistant" in url.lower():
            teacher_type = "專任助理教授"
        elif "lecturer" in url.lower():
            teacher_type = "專任講師"
        else:
            teacher_type = "專任教師"

    try:
        response = requests.get(url, headers=headers, timeout=30)
        response.encoding = 'utf-8'

        if response.status_code != 200:
            continue

        soup = BeautifulSoup(response.text, 'lxml')

        main_content = soup.select_one('#main-content') or soup.select_one('main') or soup

        tables = main_content.find_all('table')

        for table in tables:
            rows = table.find_all('tr')
            if not rows:
                continue

            header_row = rows[0]
            header_cells = header_row.find_all(['th', 'td'])
            headers = [cell.text.strip() for cell in header_cells]

            has_name_column = any('姓名' in header for header in headers)

            if has_name_column:
                name_idx = -1
                field_idx = -1

                for i, header in enumerate(headers):
                    if '姓名' in header:
                        name_idx = i
                    if any(term in header for term in ['研究領域', '專長', '研究方向']):
                        field_idx = i

                for row in rows[1:]:
                    cells = row.find_all('td')

                    if len(cells) <= max(name_idx, field_idx if field_idx >= 0 else 0):
                        continue

                    name = cells[name_idx].text.strip() if name_idx >= 0 else ""
                    research_field = cells[field_idx].text.strip() if field_idx >= 0 and field_idx < len(cells) else "未找到"

                    name_match = re.search(r'([\u4e00-\u9fa5]{2,4})', name)
                    if name_match:
                        clean_name = name_match.group(1).strip()
                        if len(clean_name) >= 2:
                            name = clean_name

                    if name and not re.match(r'^(教授|副教授|助理教授|講師|系主任|榮譽教授)$', name):
                        all_teachers.append({
                            "類型": teacher_type,
                            "姓名": name,
                            "研究領域": research_field
                        })

        if not [t for t in all_teachers if t["類型"] == teacher_type]:
            page_text = soup.get_text()

            name_matches = re.finditer(r'姓\s*名[：:]\s*([\u4e00-\u9fa5]{2,4})', page_text)
            for match in name_matches:
                name = match.group(1).strip()

                context_start = max(0, match.start() - 50)
                context_end = min(len(page_text), match.end() + 300)
                context = page_text[context_start:context_end]

                field_match = re.search(r'研究領域[：:]\s*(.*?)(?:\n|\r|。|；|$)', context)
                research_field = field_match.group(1).strip() if field_match else "未找到"

                all_teachers.append({
                    "類型": teacher_type,
                    "姓名": name,
                    "研究領域": research_field
                })

        if not [t for t in all_teachers if t["類型"] == teacher_type]:
            cards = main_content.select('.teacher-card, .profile, .faculty-container > div, .listBS > div')

            for card in cards:
                name_elem = card.select_one('h3, h4, strong, b, .name')

                if name_elem:
                    name = name_elem.text.strip()
                    name_match = re.search(r'([\u4e00-\u9fa5]{2,4})', name)
                    if name_match:
                        clean_name = name_match.group(1).strip()

                        card_text = card.text
                        field_match = re.search(r'研究領域[：:]\s*(.*?)(?:\n|\r|。|；|$)', card_text)
                        research_field = field_match.group(1).strip() if field_match else "未找到"

                        all_teachers.append({
                            "類型": teacher_type,
                            "姓名": clean_name,
                            "研究領域": research_field
                        })

        if not [t for t in all_teachers if t["類型"] == teacher_type]:
            paragraphs = main_content.find_all(['p', 'div', 'li'])

            for p in paragraphs:
                p_text = p.text.strip()
                if len(p_text) < 10:
                    continue

                teacher_matches = re.finditer(r'([\u4e00-\u9fa5]{2,4})\s*(?:教授|副教授|助理教授|講師)', p_text)

                for match in teacher_matches:
                    name = match.group(1).strip()

                    if name and not re.match(r'^(教授|副教授|助理教授|講師|系主任|榮譽教授)$', name):
                        field_match = re.search(r'研究領域[：:]\s*(.*?)(?:\n|\r|。|；|$)', p_text)
                        research_field = field_match.group(1).strip() if field_match else "未找到"

                        all_teachers.append({
                            "類型": teacher_type,
                            "姓名": name,
                            "研究領域": research_field
                        })

    except Exception as e:
        continue

unique_teachers = []
seen_names = set()

for teacher in all_teachers:
    if teacher["姓名"] not in seen_names:
        seen_names.add(teacher["姓名"])
        unique_teachers.append(teacher)

all_teachers = unique_teachers

if all_teachers:
    df = pd.DataFrame(all_teachers)
    df = df.sort_values(by=['類型', '姓名'])

    for _, row in df.iterrows():
        print(f"{row['類型']} - {row['姓名']} - {row['研究領域']}")

    file_name = 'asia_university_csie_teachers.txt'
    with open(file_name, 'w', encoding='utf-8') as f:
        for _, row in df.iterrows():
            f.write(f"{row['類型']} - {row['姓名']} - {row['研究領域']}\n")

    files.download(file_name)
else:
    print("未找到任何教師資料")

兼任教師 - 劉育寰 - 個人通信網路、行動計算、系統模擬
兼任教師 - 吳怡德 - 資訊安全，資料安全技術，資安產品開發
兼任教師 - 周存貹 - 醫療資訊、醫療影像、程式設計、儀器連線
兼任教師 - 廖益聖 - 機器學習、臨床神經學研究
兼任教師 - 張嘉淵 - 行動通訊、資訊安全、密碼學、資訊網路與網路安全
兼任教師 - 張峯銘 - 小資料學習、人工智慧、資料探勘、模糊類神經、網際網路、程式語言、彈性製造系統
兼任教師 - 徐中行 - Software methods (including algorithms, compilers, systems software, and tools) that can enable the synergistic use of many dynamic, heterogeneous hardware resources for enhancing quality of services such as performance, energy efficiency,pro
兼任教師 - 曹世昌 - 《RFID 原理》、《應用與實施》
兼任教師 - 李正吉 - 資訊安全、區塊鏈應用、精準健康、智慧醫療、工控安全
兼任教師 - 林一平 - 個人通信網路、行動計算、系統模擬
兼任教師 - 林詠章 - 資訊安全、區塊鏈應用、精準健康、智慧醫療、工控安全
兼任教師 - 梁文隆 - 光學工程、白光照明、顯示技術、生醫光電、太陽能光電、虛擬實境
兼任教師 - 温在昇 - 智慧化系統的應用、節能系統、碳足跡管理、永續管理
兼任教師 - 許健 - 電子電路、微電子、產品研發、積體電路
兼任教師 - 陳榮燊 - 光學工程、白光照明、顯示技術、生醫光電、太陽能光電、虛擬實境
兼任教師 - 陳義吉 - 機器學習、臨床神經學研究
兼任教師 - 高興全 - 網站設計、網站企劃、網站攝影企劃、品牌設計、品牌策略
兼任教師 - 黃麒昌 - 電子電路、微電子、產品研發、積體電路
專任副教授 - 朱學亭 - 生物資訊、數位學習、圖型識別、類神經網路、人工智慧
專任副教授 - 柯賢儒 - 自動控制、訊號處理、電腦網路
專任副教授 - 王經篤 - 雲端計算 、文件資訊擷取 、網路資訊擷取 、生物資訊
專任副教授 - 葉榮輝 - 太陽能電池、發光二極體、半導體雷

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>